In [138]:
!pip install replicate

In [139]:
!export REPLICATE_API_TOKEN=YOUR_API_TOKEN

In [140]:
import replicate
import pandas as pd

In [141]:
from replicate.client import Client

replicate = Client(api_token="YOUR_API_TOKEN")

In [142]:
train_path = 'path/to/train.csv'
questions = pd.read_csv(train_path)

In [157]:
outputs = []

In [158]:
for index, q in questions.iterrows():
    prompt = f'''<<SYS>>\n Answer the following question with A, B, C, D, or E. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>>\n\n</s> 
    [INST] Question: {q['prompt']} \n 
    A: {q['A']}\n
    B: {q['B']}\n
    C: {q['C']}\n
    D: {q['D']}\n
    E: {q['E']}\n [/INST] '''
    output = replicate.run(
    "meta/llama-2-7b-chat:13c3cdee13ee059ab779f0291d29054dab00a47dad8261375654de5540165fb0",
    input={
        "prompt":prompt,
        'seed':42
    }
    )
    outputs.append("".join([i for i in output]))

In [159]:
import re

def extract_answer(text):
    text = text.strip()
    if len(text) == 1:
        return text
    match = re.search(r'\b([ABCDE]):|\(([ABCDE])\)|\b([ABCDE]).|\b([ABCDE])\)', text)
    if match:
        # Returns the first non-None group
        return next(group for group in match.groups() if group is not None)
    else:
        return "No Answer Found"

In [160]:
answers = []
for output in outputs:
    answer = extract_answer(output)
    answers.append(answer)

In [167]:
#save answers to csv
answers_df = pd.DataFrame(answers)
answers_df.to_csv('llama-2-7b-answers.csv')